In [2]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)

net(X)

tensor([[-0.1162,  0.0040,  0.0856,  0.0443,  0.1198, -0.1034,  0.0087,  0.0161,
         -0.3117,  0.0102],
        [-0.0211,  0.0719,  0.0476,  0.0228,  0.0640, -0.1122,  0.1205,  0.1089,
         -0.3999,  0.0659]], grad_fn=<AddmmBackward0>)

In [3]:
#nn.Sequential是一个模组，我们也可以自己定义一个模组来构建神经网络
class MLP(nn.Module):   # 表示是nn模组的子类
    def __init__(self):   # 定义有哪些参数
        super().__init__()  # 调用一个父类，来先设置需要的内部的参数
        self.hidden = nn.Linear(20, 256)   # 定义一个线下层存在一个成员变量中
        self.out = nn.Linear(256,10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

net = MLP()
net(X)
    

tensor([[-0.0999,  0.0850,  0.0518, -0.0136,  0.1239, -0.1613, -0.0715,  0.1913,
          0.2316, -0.0958],
        [ 0.0376,  0.0959,  0.0873, -0.0294,  0.0946, -0.0899, -0.0307,  0.0194,
          0.0905, -0.0806]], grad_fn=<AddmmBackward0>)

In [4]:
## 如何实现前面的nn.Sequential模组
class MySequential(nn.Module):
    def __init__(self, *args):  #*args表示接入一个有序列表, **表示字典
        super().__init__()
        for block in args:
            self._modules[block] = block  # 是一个有序的字典,_modules是一个特殊的容器，表示放进去的是每一层网络

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)    #利用每一层网络处理输入X
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0338, -0.0694,  0.1117,  0.1536, -0.0029, -0.0604,  0.0495, -0.1078,
         -0.2826,  0.1289],
        [ 0.0733, -0.0381,  0.0374,  0.1333,  0.0127, -0.0720,  0.0955, -0.0434,
         -0.3123,  0.0732]], grad_fn=<AddmmBackward0>)

In [5]:
# 当Sequential这个类无法满足我们的计算需求时,我们可以自定义类来实现特殊的计算
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.3965, grad_fn=<SumBackward0>)

In [6]:
## 可以嵌套使用各种模组中的子类
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
        
    def forward(self, X):
        return self.linear(self.net(X))
        
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.2316, grad_fn=<SumBackward0>)